In [3]:
import numpy as np
# soft threshold function which is useful in our lasso path.
def soft_threshold(b, gam):
    return np.sign(b) * np.maximum(np.abs(b) - gam, 0.0)

In [9]:
def lasso_path(
    X, y,
    n_lambdas=100,
    lambda_min_ratio=1e-3,
    tol=1e-6,
    max_iter=10_000,
):

    X = np.asarray(X, float)
    y = np.asarray(y, float)
    n, p = X.shape

    # Means for intercept update
    x_means = X.mean(axis=0)
    y_mean  = y.mean()

    # Max Lambda Value: lambda_max = (1/n) max_j |x_j^T (y - y_mean)|
    # This is used to set the lambda range. glmnet also does the same thing
    yc = y - y_mean
    lambda_max = max(np.max(np.abs(X.T @ yc)) / n, 1e-12)
    lambdas = np.geomspace(lambda_max, lambda_max * lambda_min_ratio, n_lambdas)

    #Initialize the betas and intercepts
    betas = np.zeros((len(lambdas), p))
    intercepts = np.zeros(len(lambdas))

    # Warm starts
    beta = np.zeros(p)
    beta0 = y_mean
    Xbeta = X @ beta   # Compute X @ beta for zj and it will be updated in the next loop.

    for t, lam in enumerate(lambdas):  #Loop for each lambda
        prev_beta0 = beta0

        for _ in range(max_iter):  # For each lambda, we do the same 
            max_change = 0.0

            for j in range(p):
                xj = X[:, j]
                denom = float(xj @ xj)          # ||x_j||^2

                # zj calculation based on formula
                zj = y - beta0 - Xbeta + xj * beta[j]
                beta_hat = float(zj @ xj) / denom
                gamma = (n * lam) / denom
                new_bj = soft_threshold(beta_hat, gamma)

                delta = new_bj - beta[j]
                beta[j] = new_bj
                Xbeta += xj * delta             # update X@beta
                if abs(delta) > max_change:     # Change in coefficient only keep the maximum and compare it to tolerance.
                    max_change = abs(delta)

            # Intercept beta0 calculation
            beta0 = y_mean - float(x_means @ beta)
            # Change in intercept same as Change in coefficient.
            if abs(beta0 - prev_beta0) > max_change:
                max_change = abs(beta0 - prev_beta0)
            prev_beta0 = beta0

            if max_change < tol:   #stop when reach tolerance
                break

        betas[t] = beta
        intercepts[t] = beta0

    return betas, intercepts, lambdas


In [21]:
rng = np.random.default_rng(123)
n = 1000
p = 3  # include extra noise features so LASSO can select
X = rng.standard_normal((n, p))
true_beta = np.zeros(p)
true_beta[0] = 3.0     # x1
true_beta[1] = -17.0   # x2
true_beta[2] = 5.0     # x3
eps = rng.normal(0, 1, size=n)
y = X @ true_beta + eps

In [23]:
betas, intercepts, lambdas = lasso_path(
    X, y
)


In [31]:
betas

array([[  0.        ,  -0.        ,   0.        ],
       [  0.        ,  -1.13875076,   0.        ],
       [  0.        ,  -2.20075353,   0.        ],
       [  0.        ,  -3.19118086,   0.        ],
       [  0.        ,  -4.11485671,   0.        ],
       [  0.        ,  -4.9762799 ,   0.        ],
       [  0.        ,  -5.77964605,   0.        ],
       [  0.        ,  -6.528868  ,   0.        ],
       [  0.        ,  -7.22759491,   0.        ],
       [  0.        ,  -7.87922996,   0.        ],
       [  0.        ,  -8.48694699,   0.        ],
       [  0.        ,  -9.05370593,   0.        ],
       [  0.        ,  -9.58226721,   0.        ],
       [  0.        , -10.07520523,   0.        ],
       [  0.        , -10.53492087,   0.        ],
       [  0.        , -10.96365322,   0.        ],
       [  0.        , -11.36349045,   0.        ],
       [  0.        , -11.7350285 ,   0.1968336 ],
       [  0.        , -12.0805096 ,   0.5284504 ],
       [  0.        , -12.40270

In [33]:
lambdas

array([17.10488288, 15.95207102, 14.87695482, 13.87429786, 12.93921662,
       12.06715672, 11.25387075, 10.49539752,  9.78804286,  9.12836153,
        8.51314051,  7.93938334,  7.40429547,  6.90527074,  6.4398786 ,
        6.00585234,  5.60107799,  5.22358408,  4.871532  ,  4.54320704,
        4.23701009,  3.95144979,  3.6851353 ,  3.43676952,  3.20514275,
        2.98912686,  2.78766971,  2.59979011,  2.42457295,  2.26116485,
        2.10876991,  1.96664587,  1.83410052,  1.71048829,  1.5952071 ,
        1.48769548,  1.38742979,  1.29392166,  1.20671567,  1.12538707,
        1.04953975,  0.97880429,  0.91283615,  0.85131405,  0.79393833,
        0.74042955,  0.69052707,  0.64398786,  0.60058523,  0.5601078 ,
        0.52235841,  0.4871532 ,  0.4543207 ,  0.42370101,  0.39514498,
        0.36851353,  0.34367695,  0.32051428,  0.29891269,  0.27876697,
        0.25997901,  0.2424573 ,  0.22611649,  0.21087699,  0.19666459,
        0.18341005,  0.17104883,  0.15952071,  0.14876955,  0.13

In [67]:
intercepts

array([-6.74323543e-01, -6.38626246e-01, -6.05334828e-01, -5.74287140e-01,
       -5.45331962e-01, -5.18328266e-01, -4.93144529e-01, -4.69658092e-01,
       -4.47754562e-01, -4.27327256e-01, -4.08276683e-01, -3.90510054e-01,
       -3.73940837e-01, -3.58488329e-01, -3.44077269e-01, -3.30637466e-01,
       -3.18103461e-01, -3.00362006e-01, -2.79264083e-01, -2.59588088e-01,
       -2.41238190e-01, -2.24125014e-01, -2.08165208e-01, -1.93281040e-01,
       -1.79400015e-01, -1.66454525e-01, -1.54224331e-01, -1.42489745e-01,
       -1.31546030e-01, -1.21339885e-01, -1.11821600e-01, -1.02944815e-01,
       -9.46662966e-02, -8.69457222e-02, -7.97454887e-02, -7.30305268e-02,
       -6.67681308e-02, -6.09277994e-02, -5.54810868e-02, -5.04014644e-02,
       -4.56641915e-02, -4.12461950e-02, -3.71259566e-02, -3.32834085e-02,
       -2.96998353e-02, -2.63577830e-02, -2.32409738e-02, -2.03342270e-02,
       -1.76233853e-02, -1.50952453e-02, -1.27374934e-02, -1.05386461e-02,
       -8.48799379e-03, -

In [65]:
import pandas as pd

X_df = pd.DataFrame(X, columns=[f"x{i+1}" for i in range(p)])
y_df = pd.DataFrame({"y": y})

# Save separately
X_df.to_csv("X_data.csv", index=False)
y_df.to_csv("y_data.csv", index=False)

After I run the glmnet code in R. I got the similar results. The differences between my code and glmnet are very small(about 0.001%), so I can say you get the same solution path as the glmnet program by Hastie, Tibshirani and Friedman. The R code is the other file in my submission. The glmnet code only output 77 lambda coefficients, even though I set nlambda 100. This is because if λ gets too small, glmnet shortens the sequence automatically. But we can still see that the output are nearly same.

In [70]:
glmnet_result = pd.read_csv("glmnet_coefficients.csv")

In [74]:
print(glmnet_result)

  Unnamed: 0  lambda_1  lambda_2  lambda_3  lambda_4  lambda_5  lambda_6  \
0  Intercept -0.674324 -0.638626 -0.605335 -0.574287 -0.545332 -0.518328   
1         x1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2         x2  0.000000 -1.138751 -2.200754 -3.191181 -4.114857 -4.976280   
3         x3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

   lambda_7  lambda_8  lambda_9  ...  lambda_68  lambda_69  lambda_70  \
0 -0.493145 -0.469658 -0.447755  ...   0.009925   0.010596   0.011222   
1  0.000000  0.000000  0.000000  ...   2.825930   2.837224   2.847758   
2 -5.779646 -6.528868 -7.227595  ... -16.811318 -16.822295 -16.832533   
3  0.000000  0.000000  0.000000  ...   4.906374   4.916259   4.925478   

   lambda_71  lambda_72  lambda_73  lambda_74  lambda_75  lambda_76  lambda_77  
0   0.011806   0.012351   0.012859   0.013333   0.013774   0.014187   0.014571  
1   2.857581   2.866743   2.875287   2.883255   2.890686   2.897617   2.904080  
2 -16.8420